# Single EV Snapshot Experiments (Voltage-Focused)

This notebook builds on the baseline OPF and performs snapshot EV stress tests.

Goals:
1. Place a single EV at key locations (slack, mid-feeder, far-end, weakest bus) and observe voltage impact.
2. Increase EV severity at a fixed location to identify voltage sensitivity.
3. Compare two-EV cases: clustered vs distributed.

This notebook is snapshot-based (single operating point), not time-series.
We focus on voltage behaviour (per-unit) and do not analyse thermal congestion at this stage.


In [7]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

using PowerModelsDistribution
using DataFrames
using Ipopt
using JuMP
using LinearAlgebra
using Plots
using Graphs
using CairoMakie
using Statistics

const PMD = PowerModelsDistribution

  Activating project at `/mnt/c/Users/auc009/OneDrive - CSIRO/Documents/power-models-distribution/pmd_ev_experiments`


PowerModelsDistribution

In [8]:
# Input feeder (same as baseline)
file = "../data/networks/lv_small_feeder/Master.dss"

# Output folders
fig_dir = "../results/figures"
mkpath(fig_dir)

"../results/figures"

In [9]:
eng4w = PMD.parse_file(file, transformations=[PMD.transform_loops!])

println("Buses: ", length(get(eng4w, "bus", Dict())))
println("Lines: ", length(get(eng4w, "line", Dict())))
println("Loads: ", length(get(eng4w, "load", Dict())))

# Ensure sbase_default consistent with your baseline approach
eng4w["settings"]["sbase_default"] = 1.0*1E3/eng4w["settings"]["power_scale_factor"]

# Add absolute voltage bounds (phase + neutral)
PMD.add_bus_absolute_vbounds!(
    eng4w,
    phase_lb_pu = 0.9,
    phase_ub_pu = 1.1,
    neutral_ub_pu = 0.1
)

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 23 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 24 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 25 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 31 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line48 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : ba

Buses: 65
Lines: 64
Loads: 23


Dict{String, Any} with 10 entries:
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "name"           => "enwl_network_20_feeder_5_4wire"
  "line"           => Dict{String, Any}("line48"=>Dict{String, Any}("length"=>8…
  "conductor_ids"  => [1, 2, 3, 4]
  "settings"       => Dict{String, Any}("sbase_default"=>1.0, "vbases_default"=…
  "files"          => ["../data/networks/lv_small_feeder/Master.dss", "../data/…
  "load"           => Dict{String, Any}("load2"=>Dict{String, Any}("source_id"=…
  "bus"            => Dict{String, Any}("32"=>Dict{String, Any}("rg"=>Float64[]…
  "linecode"       => Dict{String, Any}("lc8"=>Dict{String, Any}("b_fr"=>[0.0 0…
  "data_model"     => ENGINEERING

## Build topology objects (graph, electrical distance, fixed layout)

We build:
- a bus-name graph from line connectivity
- electrical distance from the source bus (weighted shortest path)
- a fixed 2D layout for topology plotting

The layout is computed once and reused for all scenarios so plots are directly comparable.

In [10]:
function build_lines_df(eng)
    line_dict = get(eng, "line", Dict())
    rows = NamedTuple[]

    for (id, ln) in line_dict
        bus1 = get(ln, "bus1", get(ln, "f_bus", nothing))
        bus2 = get(ln, "bus2", get(ln, "t_bus", nothing))
        if bus1 === nothing || bus2 === nothing
            continue
        end

        # OpenDSS style: "busname.1.2.3.4" -> take base name for topology
        b1 = split(String(bus1), ".")[1]
        b2 = split(String(bus2), ".")[1]

        # length is often available; units depend on DSS. We'll treat missing as 1.0.
        L = get(ln, "length", 1.0)
        push!(rows, (Bus1=b1, Bus2=b2, Length=float(L)))
    end

    DataFrame(rows)
end

lines_df = build_lines_df(eng4w)
first(lines_df, 5)


Row,Bus1,Bus2,Length
,SubStrin…,SubStrin…,Float64
1,41,49,8.0751
2,8,9,0.093236
3,24,25,0.228
4,38,39,0.12075
5,6,7,0.081934


## Plot Helpers

In [11]:
"Build a bus-name -> index mapping and Graph + edge length lookup."
function build_graph_and_weights(lines_df::DataFrame)
    buses = unique(vcat(lines_df.Bus1, lines_df.Bus2))
    sort!(buses)
    b2i = Dict(b => i for (i,b) in enumerate(buses))
    i2b = Dict(i => b for (b,i) in b2i)

    g = Graph(length(buses))
    w = Dict{Tuple{Int,Int}, Float64}()

    for r in eachrow(lines_df)
        i = b2i[r.Bus1]
        j = b2i[r.Bus2]
        add_edge!(g, i, j)
        # store weight both directions
        w[(i,j)] = r.Length
        w[(j,i)] = r.Length
    end

    return g, w, b2i, i2b
end

"Weighted shortest-path distances from a source index (Dijkstra)."
function dijkstra_distances(g::Graph, w::Dict{Tuple{Int,Int},Float64}, s::Int)
    n = nv(g)
    dist = fill(Inf, n)
    prev = fill(0, n)
    visited = falses(n)

    dist[s] = 0.0
    pq = [(0.0, s)]  # (dist,node)

    while !isempty(pq)
        sort!(pq, by=x->x[1])
        (d,u) = popfirst!(pq)
        if visited[u]; continue; end
        visited[u] = true

        for v in neighbors(g, u)
            nd = d + get(w, (u,v), 1.0)
            if nd < dist[v]
                dist[v] = nd
                prev[v] = u
                push!(pq, (nd, v))
            end
        end
    end

    return dist, prev
end

g, w, b2i, i2b = build_graph_and_weights(lines_df)


(SimpleGraph{Int64}(64, [[12, 65], [3, 64], [2, 4], [3, 5], [4, 6], [5, 7], [6, 8], [7, 9], [8, 10], [9, 11]  …  [45, 62], [36], [36], [36], [36], [36], [56, 63], [62, 64], [2, 63], [1]]), Dict((48, 36) => 8.0751, (11, 10) => 0.057454, (1, 12) => 0.68677, (1, 65) => 1.0, (23, 34) => 0.064762, (25, 26) => 0.51966, (7, 8) => 0.05331, (35, 36) => 2.8922, (55, 36) => 8.0751, (36, 50) => 8.0751…), Dict{SubString{String}, Int64}("32" => 26, "29" => 22, "1" => 1, "12" => 4, "54" => 50, "20" => 13, "9" => 64, "2" => 12, "6" => 56, "41" => 36…), Dict{Int64, SubString{String}}(5 => "13", 56 => "6", 16 => "23", 20 => "27", 55 => "59", 35 => "40", 60 => "63", 30 => "36", 19 => "26", 32 => "38"…))

In [12]:
# Source bus name in eng model is typically "sourcebus"
# We will use it as the topology reference bus.
source_name = "sourcebus"
@assert haskey(b2i, source_name) "Source bus name '$source_name' not found in topology buses. Check bus naming."

source_idx = b2i[source_name]
dist_raw, prev = dijkstra_distances(g, w, source_idx)

# Convert to "km-like" scale if your DSS lengths were meters.
# If your line lengths are already in km, set scale = 1.0.
scale_to_km = 1.0/1000
dist_km = dist_raw .* scale_to_km

println("Max electrical distance (km): ", maximum(dist_km[isfinite.(dist_km)]))


Max electrical distance (km): 0.043825537


In [13]:
"Build children list from predecessor array (shortest-path tree)."
function tree_children(prev::Vector{Int})
    n = length(prev)
    children = [Int[] for _ in 1:n]
    for v in 1:n
        p = prev[v]
        if p != 0
            push!(children[p], v)
        end
    end
    return children
end

"Assign y positions by DFS order; x = electrical distance."
function tree_layout(prev::Vector{Int}, dist_km::Vector{Float64}, root::Int)
    children = tree_children(prev)
    y = fill(0.0, length(prev))
    next_y = Ref(0.0)

    function dfs(u::Int)
        if isempty(children[u])
            y[u] = next_y[]
            next_y[] += 1.0
        else
            for c in sort(children[u])
                dfs(c)
            end
            y[u] = mean(y[children[u]])
        end
    end

    dfs(root)
    x = copy(dist_km)
    return x, y
end

xpos, ypos = tree_layout(prev, dist_km, source_idx)


([0.001, 0.0040497, 0.0040872899999999995, 0.004134702999999999, 0.0041745279999999985, 0.004219125999999998, 0.006236425999999998, 0.006289735999999998, 0.006335833999999998, 0.006377892999999997  …  0.001926944, 0.043041636999999994, 0.043825537, 0.043041636999999994, 0.043825537, 0.043041636999999994, 0.0020088780000000004, 0.002075164, 0.0021684, 0.0], [11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0  …  11.0, 18.0, 19.0, 20.0, 21.0, 22.0, 11.0, 11.0, 11.0, 11.0])

In [14]:
function plot_topology_with_markers(
    g::Graph, xpos::Vector{Float64}, ypos::Vector{Float64}, dist_km::Vector{Float64};
    source_idx::Int,
    ev_indices::Vector{Int}=Int[],
    title_str::String="LV Topology (colored by electrical distance)",
    save_path::Union{Nothing,String}=nothing
)
    fig = Figure(size=(1000, 700))
    ax = Axis(fig[1,1], title=title_str)

    # edges
    for e in edges(g)
        u = src(e); v = dst(e)
        lines!(ax, [xpos[u], xpos[v]], [ypos[u], ypos[v]], color=:gray50, linewidth=2)
    end

    # nodes colored by distance
    scatter!(ax, xpos, ypos, color=dist_km, markersize=10)

    # source marker
    scatter!(ax, [xpos[source_idx]], [ypos[source_idx]], markersize=22, color=:black)
    text!(ax, "source", position=(xpos[source_idx], ypos[source_idx] + 0.6), align=(:center, :bottom))

    # EV markers
    for (k, ev) in enumerate(ev_indices)
        scatter!(ax, [xpos[ev]], [ypos[ev]], markersize=22, color=:red)
        text!(ax, "EV$(k)", position=(xpos[ev], ypos[ev] + 0.6), align=(:center, :bottom))
    end

    hidedecorations!(ax)  # keeps it clean like your screenshot
    hidespines!(ax)

    if save_path !== nothing
        save(save_path, fig)
    end
    return fig
end


plot_topology_with_markers (generic function with 1 method)

## Transform to OPF model and run Baseline OPF

We convert the engineering network to a mathematical model suitable for OPF.
We keep full four-wire physics (no Kron reduction) and no phase projection.

We then solve the baseline snapshot OPF and store voltages to:
- identify the weakest bus (minimum phase voltage)
- serve as the reference case for all EV scenarios

In [15]:
math4w_base = PMD.transform_data_model(
    eng4w;
    multinetwork=false,
    kron_reduce=false,
    phase_project=false
)

println("Running baseline unbalanced OPF...")
res_base = PMD.solve_mc_opf(math4w_base, IVRENPowerModel, Ipopt.Optimizer)
println("Baseline status: ", res_base["termination_status"])

Running baseline unbalanced OPF...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:     8688
Number of nonzeros in inequality constraint Jacobian.:     1944
Number of nonzeros in Lagrangian Hessian.............:     2882

The Jacobian for the equality constraints contains an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations